In [1]:
import torch
import config
from utils import (
    get_model_instance_segmentation,
    collate_fn,
    get_transform,
    myOwnDataset,
)

print("Torch version:", torch.__version__)


Torch version: 2.1.2+cu121


In [2]:
# create own Dataset
my_dataset = myOwnDataset(
    root=config.train_data_dir, annotation=config.train_coco, transforms=get_transform()
)

# own DataLoader
data_loader = torch.utils.data.DataLoader(
    my_dataset,
    batch_size=config.train_batch_size,
    shuffle=config.train_shuffle_dl,
    num_workers=config.num_workers_dl,
    collate_fn=collate_fn,
)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [3]:
# select device (whether GPU or CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# DataLoader is iterable over Dataset
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)


model = get_model_instance_segmentation(config.num_classes)

# move model to the right device
model.to(device)

# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params, lr=config.lr, momentum=config.momentum, weight_decay=config.weight_decay
)

len_dataloader = len(data_loader)

[{'boxes': tensor([[214.0000, 379.0000, 254.5000, 424.5000]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([21], device='cuda:0'), 'area': tensor([1842.7500], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[209.0000, 272.0000, 266.5000, 378.5000],
        [341.0000, 273.0000, 398.5000, 385.0000]], device='cuda:0'), 'labels': tensor([1, 1], device='cuda:0'), 'image_id': tensor([18], device='cuda:0'), 'area': tensor([6123.7500, 6440.0000], device='cuda:0'), 'iscrowd': tensor([0, 0], device='cuda:0')}]
[{'boxes': tensor([[125.0000, 407.0000, 287.0000, 585.5000]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([8], device='cuda:0'), 'area': tensor([28917.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[211.0000, 259.0000, 275.5000, 398.0000],
        [342.0000, 258.0000, 409.5000, 392.5000]], device='cuda:0'), 'labels': tensor([1, 1], device='cuda:0'), 'image_id':

In [5]:
# Training
for epoch in range(config.num_epochs):
    print(f"Epoch: {epoch}/{config.num_epochs}")
    model.train()
    i = 0
    for imgs, annotations in data_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        # Debug
        # print(annotations)
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        print(f"Iteration: {i}/{len_dataloader}, Loss: {losses}")

Epoch: 0/10
Iteration: 1/48, Loss: 0.13282938301563263
Iteration: 2/48, Loss: 0.21864429116249084
Iteration: 3/48, Loss: 0.17990145087242126
Iteration: 4/48, Loss: 0.17350158095359802
Iteration: 5/48, Loss: 0.1792048215866089
Iteration: 6/48, Loss: 0.25175219774246216
Iteration: 7/48, Loss: 0.13896684348583221
Iteration: 8/48, Loss: 0.17440681159496307
Iteration: 9/48, Loss: 0.17365415394306183
Iteration: 10/48, Loss: 0.1695408970117569
Iteration: 11/48, Loss: 0.14602114260196686
Iteration: 12/48, Loss: 0.16506849229335785
Iteration: 13/48, Loss: 0.13795866072177887
Iteration: 14/48, Loss: 0.13282962143421173
Iteration: 15/48, Loss: 0.1266447752714157
Iteration: 16/48, Loss: 0.11670123785734177
Iteration: 17/48, Loss: 0.10136821866035461
Iteration: 18/48, Loss: 0.11734966933727264
Iteration: 19/48, Loss: 0.16010744869709015
Iteration: 20/48, Loss: 0.3185419738292694
Iteration: 21/48, Loss: 0.21407993137836456
Iteration: 22/48, Loss: 0.1726917177438736
Iteration: 23/48, Loss: 0.15309788